In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [1]:
import requests

url = "https://chm.iiserb.ac.in/faculty"  # replace with real page
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers, verify=False)
html = response.text

print(html[:500])

<!--// Set XSS protection (legacy)-->

<style>
.MultiCarousel {
	float: left;
	overflow: hidden;
	padding: 15px;
	width: 100%;
	position: relative;
}
.MultiCarousel .MultiCarousel-inner {
	transition: 1s ease all;
	float: left;
}
.MultiCarousel .MultiCarousel-inner .item {
	float: left;
}
.MultiCarousel .MultiCarousel-inner .item > div {
	text-align: center;
	padding: 10px;
	margin: 10px;
	background: #EFF0F5;
	color: #E4E4E4;
}
.MultiCarousel .leftLst,
.MultiCarousel .r


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chm.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [4]:
import requests
from bs4 import BeautifulSoup
import csv
import time

BASE_URL = "https://chm.iiserb.ac.in"  # main site
FACULTY_PAGE = f"{BASE_URL}/Home/Faculty"

headers = {"User-Agent": "Mozilla/5.0"}

# Use verify=False if SSL gives errors
response = requests.get(FACULTY_PAGE, headers=headers, verify=False)
soup = BeautifulSoup(response.text, "html.parser")

faculty_blocks = soup.select("div#myTabContent div.col-md-3.prdct-grid")

with open("iiserb_faculty.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Name", "Designation", "Email", "Profile Link", "PhD", "Postdoc"])

    for block in faculty_blocks:
        # Name
        name_tag = block.select_one("div.product-name strong")
        name = name_tag.text.strip() if name_tag else ""

        # Profile link
        profile_tag = block.select_one("div.product-name a[href^='http']")
        profile_link = profile_tag.get("href").strip() if profile_tag else ""

        # Other info (designation, email)
        p_tag = block.select_one("div.product-name p")
        text_lines = [t.strip() for t in p_tag.get_text(separator="\n").split("\n") if t.strip()] if p_tag else []
        designation = text_lines[0] if len(text_lines) > 0 else ""
        email = next((t for t in text_lines if "@" in t), "")

        # Initialize
        phd = ""
        postdoc = ""

        if profile_link:
            try:
                prof_resp = requests.get(profile_link, headers=headers, verify=False)
                prof_soup = BeautifulSoup(prof_resp.text, "html.parser")
                li_tags = prof_soup.select("div.row.clearfix li")

                for li in li_tags:
                    text = li.get_text(strip=True)
                    if "Ph.D" in text or "PhD" in text:
                        phd = text
                    elif "Postdoctoral" in text or "Postdoc" in text:
                        postdoc = text

                time.sleep(0.5)  # polite delay
            except Exception as e:
                print(f"Failed to fetch {profile_link}: {e}")

        writer.writerow([name, designation, email, profile_link, phd, postdoc])

print("Scraping completed! Data saved to iiserb_faculty.csv")


Scraping completed! Data saved to iiserb_faculty.csv


/home/codespace/.local/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'chm.iiserb.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
